In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Natal,BR,2021-04-18 00:11:29,-5.7950,-35.2094,78.80,89,75,3.44
1,1,Hof,DE,2021-04-18 00:14:40,50.3167,11.9167,37.99,93,100,4.00
2,2,Port Alfred,ZA,2021-04-18 00:12:30,-33.5906,26.8910,65.28,60,34,12.50
3,3,Hluti,SZ,2021-04-18 00:14:41,-27.2167,31.6167,51.10,87,40,3.51
4,4,Khromtau,KZ,2021-04-18 00:14:41,50.2503,58.4347,48.22,76,29,8.90


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]

# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.6)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of humidity
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the humidity %.
humidity = city_data_df["Humidity"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.6)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of % cloudiness
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the humidity %.
clouds = city_data_df["Cloudiness"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.6)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of Wind Speed
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the humidity %.
wind_speed = city_data_df["Wind Speed"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.6)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=wind_speed, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [11]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]


In [12]:
preferred_cities_df.count()

City_ID       214
City          214
Country       210
Date          214
Lat           214
Lng           214
Max Temp      214
Humidity      214
Cloudiness    214
Wind Speed    214
dtype: int64

In [13]:
preferred_cities_df = preferred_cities_df.dropna(subset=['Country'])

In [14]:
preferred_cities_df.count()

City_ID       210
City          210
Country       210
Date          210
Lat           210
Lng           210
Max Temp      210
Humidity      210
Cloudiness    210
Wind Speed    210
dtype: int64

In [15]:
# Create a DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Natal,BR,78.80,-5.7950,-35.2094,
6,Arinos,BR,70.97,-15.9169,-46.1056,
7,Hambantota,LK,77.18,6.1241,81.1185,
9,Rikitea,PF,77.58,-23.1203,-134.9692,
16,Doha,QA,78.80,25.2867,51.5333,
17,Carupano,VE,77.58,10.6678,-63.2585,
18,Redmond,US,80.01,47.6740,-122.1215,
23,Merauke,ID,76.41,-8.4667,140.3333,
28,Olinda,BR,75.20,-8.0089,-34.8553,
30,Alta Floresta,BR,71.15,-9.8756,-56.0861,


In [21]:
# Set parameters to search for a hotel.
params = {"radius" : 5000, "type" : "lodging", "key" : g_key}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][1]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [28]:
hotel_df.head(40)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Natal,BR,78.80,-5.7950,-35.2094,Arituba Park Hotel
6,Arinos,BR,70.97,-15.9169,-46.1056,pousada central
7,Hambantota,LK,77.18,6.1241,81.1185,The Bungalow Hambantota
9,Rikitea,PF,77.58,-23.1203,-134.9692,Pension Bianca & Benoit
16,Doha,QA,78.80,25.2867,51.5333,Ezdan Hotel
17,Carupano,VE,77.58,10.6678,-63.2585,Posada Mamá Dominga
18,Redmond,US,80.01,47.6740,-122.1215,Hyatt House Seattle/Redmond
23,Merauke,ID,76.41,-8.4667,140.3333,Merauke
28,Olinda,BR,75.20,-8.0089,-34.8553,Pousada Sitio Do Carmo
30,Alta Floresta,BR,71.15,-9.8756,-56.0861,Hotel Estoril


In [33]:
testing_df = hotel_df[hotel_df["Hotel Name"].astype(bool)]
testing_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Natal,BR,78.80,-5.7950,-35.2094,Arituba Park Hotel
6,Arinos,BR,70.97,-15.9169,-46.1056,pousada central
7,Hambantota,LK,77.18,6.1241,81.1185,The Bungalow Hambantota
9,Rikitea,PF,77.58,-23.1203,-134.9692,Pension Bianca & Benoit
16,Doha,QA,78.80,25.2867,51.5333,Ezdan Hotel


In [18]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

NameError: name 'info_box_template' is not defined

In [ ]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig